### **A Comparative Analysis of Public Sentiment on Twitter towards Apple and Google Products using Natural Language Processing.**  


**Group Name: Group 5**

**Members**
1. *Rose Miriti*
2. *Isaac Wadhare*
3. *Lydia Chumba*
4. *Erick Mauti*
5. *Marilyn Akinyi*
6. *Rodgers Otieno*
7. *Samwel Ongechi*

**Technical Mentor: George Kamundia**

**Phase: Phase 3 Project**

**Deadline:**

### **Table of Content**